In [1]:
import os
import astropy.io.fits as fits
import pandas as pd
import numpy as np
from pathlib import Path, PurePath
import matplotlib.pyplot as plt
import sunpy.visualization.colormaps
import matplotlib.colors as colors
import seaborn as sns
pd.options.mode.chained_assignment = None

In [18]:
def create_lookup_8nb(nx, ny):
    """ Pre-compute the 8-connectivity lookup table. This will be shared across parallel workers.
    :param nx:
    :param ny:
    :return:
    """
    # List of relative 2D coordinates for 8-neighbour connectiviy (9-element list). 1st one is the origin pixel.
    coords_8nb = np.array([[0, 0], [-1, 0], [-1, -1], [0, -1], [1, -1], [1, 0], [1, 1], [0, 1], [-1, 1]])
    # Array of 2D coordinates for a 4096 x 4096 array. Matrix convention is kept. [rows, cols] = [y-axis, x-axis]
    coords_1d = np.arange(nx * ny)
    coordy, coordx = np.unravel_index(coords_1d, [ny, nx]) # also possible by raveling a meshgrid() output
    coords2d = np.array([coordy, coordx])
    # Create the array of 2D coordinates of 8-neighbours associated with each pixel.
    # pixel 0 has 8 neighbour + itself, pixel 1 has 8 neighbour + itself, etc...
    coords2d_8nb = coords2d[np.newaxis, ...] + coords_8nb[..., np.newaxis]
    # Handle off-edges coordinates by clipping to the edges, operation done in-place. Here, square detector assumed.
    # to per-axis clipping if that ever changes for another instrument.
    np.clip(coords2d_8nb, 0, nx-1, out=coords2d_8nb)
    # Convert to 1D coordinates.
    lookup_coords = np.array([coords2d_8nb[i, 0, :] * nx + coords2d_8nb[i, 1, :] for i in range(len(coords_8nb))],
                         dtype='int32', order='C').T
    return lookup_coords


def extract_coincidentals(spikes_list, idx):
    # Spikes coordinates at given wavelength index
    spikes_w = spikes_list[idx]
    # Associated neighbour coordinates
    nb_pixels = LOOKUP_8NB[spikes_w[0, :], :]
    # Sublist of spikes data that will excludes the one serving as template
    spikes_sublist = spikes_list[:idx] + spikes_list[idx + 1:]
    # Coincidental cross-referencing.
    # mask_w_arr = np.array([np.isin(nb_pixels, index_8nb[spikes[0, :], :]).any(axis=1) for spikes in spikes_sublist])
    mask_w_arr = np.array([np.isin(nb_pixels, spikes[0, :]).any(axis=1) for spikes in spikes_sublist])
    select_pixels = mask_w_arr.any(axis=0)
    coords_w = spikes_w[0, select_pixels]
    w_tables = np.insert(mask_w_arr[:, select_pixels], idx, True, axis=0)
    # Retrieve intensity values for the selected coordinates
    intensities = spikes_w[1:, select_pixels]
    arr_w = np.concatenate([coords_w[np.newaxis, ...], intensities, w_tables], axis=0)
    arr_w = np.insert(arr_w, 3, idx, axis=0)
    return arr_w

In [3]:
data_dir = os.environ['SPIKESDATA']
spikes_db = pd.read_parquet(os.path.join(data_dir, 'spikes_df_2010.parquet'), engine='pyarrow').set_index('GroupNumber')
print(len(spikes_db))
spikes_db.head(8)

50386


,Path,Size,Time,Wavelength
GroupNumber,,,,
0,2010/07/12/2010-07-12T00h00m00.57Z_0171.spikes...,25920,2010-07-12 00:00:00.570000+00:00,171
0,2010/07/12/2010-07-12T00h00m02.07Z_0211.spikes...,37440,2010-07-12 00:00:02.070000+00:00,211
0,2010/07/12/2010-07-12T00h00m03.57Z_0304.spikes...,63360,2010-07-12 00:00:03.570000+00:00,304
0,2010/07/12/2010-07-12T00h00m05.05Z_0335.spikes...,80640,2010-07-12 00:00:05.050000+00:00,335
0,2010/07/12/2010-07-12T00h00m08.05Z_0193.spikes...,28800,2010-07-12 00:00:08.050000+00:00,193
0,2010/07/12/2010-07-12T00h00m09.59Z_0094.spikes...,69120,2010-07-12 00:00:09.590000+00:00,94
0,2010/07/12/2010-07-12T00h00m11.05Z_0131.spikes...,69120,2010-07-12 00:00:11.050000+00:00,131
1,2010/07/12/2010-07-12T00h00m12.55Z_0171.spikes...,25920,2010-07-12 00:00:12.550000+00:00,171


In [7]:
paths = spikes_db.loc[0]['Path'].apply(
    lambda s:Path(os.environ['SPIKESDATA'], s))
paths

GroupNumber
0    /mnt/data3/SDO/AIA/spikes/2010/07/12/2010-07-1...
0    /mnt/data3/SDO/AIA/spikes/2010/07/12/2010-07-1...
0    /mnt/data3/SDO/AIA/spikes/2010/07/12/2010-07-1...
0    /mnt/data3/SDO/AIA/spikes/2010/07/12/2010-07-1...
0    /mnt/data3/SDO/AIA/spikes/2010/07/12/2010-07-1...
0    /mnt/data3/SDO/AIA/spikes/2010/07/12/2010-07-1...
0    /mnt/data3/SDO/AIA/spikes/2010/07/12/2010-07-1...
Name: Path, dtype: object

In [11]:
spikes = pd.concat([pd.DataFrame(fits.getdata(f).T, columns=['coords', 'int1', 'int2']) for f in paths])
spikes.head()

,coords,int1,int2
0,10159,156,14
1,13241,221,4
2,14255,111,14
3,14361,199,11
4,17802,235,11


In [12]:
spikes[spikes['coords']==63998]

,coords,int1,int2
19,63998,49,10


In [14]:
LOOKUP_8NB = create_lookup_8nb(4096, 4096)

In [20]:
group_n = 0

In [21]:
spikes_list = [fits.getdata(f) for f in paths]
group_data = np.concatenate([extract_coincidentals(spikes_list, i) for i in range(7)], axis=1)
column_names = ['coords', 'int1', 'int2', 'wref', 'w0', 'w1', 'w2', 'w3', 'w4', 'w5', 'w6']
overlaps_df = pd.DataFrame(group_data.T, columns=column_names)
overlaps_df.insert(0, 'GroupNumber', group_n)

In [22]:
overlaps_df.head()

,GroupNumber,coords,int1,int2,wref,w0,w1,w2,w3,w4,w5,w6
0,0,142121,63,9,0,1,0,0,0,0,1,0
1,0,164350,49,0,0,1,0,0,1,0,0,0
2,0,267298,68,8,0,1,0,0,1,0,0,0
3,0,297326,73,15,0,1,0,0,0,1,0,0
4,0,349885,54,20,0,1,0,0,0,0,1,0


In [36]:
idx = 0
spikes_w = spikes_list[idx]
print(spikes_w.shape)
spikes_w[:, np.where(spikes_w[0]==63998)[0]]

(3, 4459)


array([[63998],
       [   49],
       [   10]], dtype=int32)

In [38]:
# Associated neighbour coordinates
nb_pixels = LOOKUP_8NB[spikes_w[0, :], :]
# Sublist of spikes data that will excludes the one serving as template
spikes_sublist = spikes_list[:idx] + spikes_list[idx + 1:]
# Coincidental cross-referencing.
# mask_w_arr = np.array([np.isin(nb_pixels, index_8nb[spikes[0, :], :]).any(axis=1) for spikes in spikes_sublist])
mask_w_arr = np.array([np.isin(nb_pixels, spikes[0, :]).any(axis=1) for spikes in spikes_sublist])

In [39]:
mask_w_arr.shape

(6, 4459)

In [43]:
nbpix = LOOKUP_8NB[63998, :]
nbpix

array([63998, 59902, 59901, 63997, 68093, 68094, 68095, 63999, 59903],
      dtype=int32)

In [47]:
isin = [np.isin(nbpix, spikes[0, :]) for spikes in spikes_sublist]
isin

[array([False, False, False, False, False, False, False, False, False]),
 array([False, False, False, False, False, False, False, False, False]),
 array([False, False, False, False, False, False, False, False, False]),
 array([False, False, False, False, False, False, False, False, False]),
 array([False, False, False, False, False, False, False, False, False]),
 array([False, False, False, False, False, False, False, False, False])]

In [50]:
nbpix2 = np.array([234, 32442, 59902, 324, 68093])
isin2 = np.isin(nbpix, nbpix2) 
isin2

array([False,  True, False, False,  True, False, False, False, False])

In [52]:
nbpix_x = nbpix % 4096
nbpix_y = nbpix // 4096
nbpix_x, nbpix_y

(array([2558, 2558, 2557, 2557, 2557, 2558, 2559, 2559, 2559], dtype=int32),
 array([15, 14, 14, 15, 16, 16, 16, 15, 14], dtype=int32))